## 🚀 Initialization

In [14]:
import numpy as np
import pandas as pd
from mlfluids import Fluid, set_refprop_path

set_refprop_path("/home/braun/Documents/Developer/REFPROP10")

## ⚙️ Parameters

In [15]:
MIXTURE = "H2S&CO2"

PHASE = "supercritical"
PROPS = ["PHASE", "P*", "T*", "VIS*", "TCX*"]

In [31]:
fluid = Fluid(MIXTURE)

Xpoints = 30

Tmin = max(1.05, fluid.Tmin())
Tmax = min(5, fluid.Tmax())
Tpoints = 30

Pmin = 1.05
Pmax = min(20, fluid.pmax())
Ppoints = 30

## 🧪 Data preparation

In [ ]:
X = np.linspace(0, 1, Xpoints + 1, endpoint=False)[1:]

Pr = np.geomspace(Pmin, Pmax, Ppoints)
Tr = np.geomspace(Tmin, Tmax, Tpoints)

PrTr = np.stack(np.meshgrid(Pr, Tr, indexing="ij"), axis=-1).reshape(-1, 2)

In [32]:
norm_fluid = Fluid(MIXTURE)


def calc_props_X(Xv: float) -> pd.DataFrame:
    # SETUP: NORMALIZING FLUID
    norm_fluid.set_mole_fractions([Xv, 1 - Xv])
    norm_fluid.set_state_critical()
    Fluid.set_normalizing_fluid(norm_fluid)

    # SETUP: FLUID
    fluid.set_mole_fractions([Xv, 1 - Xv])
    fluid._D = norm_fluid.rhomolar_critical()
    if PHASE:
        fluid.specify_phase(PHASE)

    # PARAMETER CALCULATION
    props_data = fluid.calc_states_props("P*", "T*", PrTr, PROPS, log_errors=True)

    print(Xv)
    print(props_data)

    return pd.DataFrame(props_data, columns=PROPS)


results = pd.concat([calc_props_X(Xv) for Xv in X], ignore_index=True)

display(results)

PT: [TPRHO error 203] Vapor iteration has not converged for T = 319.751 K and P = 43.4064 MPa.                                                                                                                                                                    
PT: [TPRHO error 203] Vapor iteration has not converged for T = 319.751 K and P = 48.0492 MPa.                                                                                                                                                                    
PT: [TPRHO error 203] Vapor iteration has not converged for T = 319.751 K and P = 53.1886 MPa.                                                                                                                                                                    
PT: [TPRHO error 203] Vapor iteration has not converged for T = 337.430 K and P = 53.1886 MPa.                                                                                                                                 

,PHASE,P*,T*,VIS*,TCX*
0,supercritical,1.05,1.05,0.619488,0.374421
1,supercritical,1.05,1.108054,0.607998,0.323185
2,supercritical,1.05,1.169318,0.618679,0.314129
3,supercritical,1.05,1.23397,0.636973,0.317541
4,supercritical,1.05,1.302196,0.659335,0.32696
...,...,...,...,...,...
26608,supercritical,20.0,4.031652,2.214411,0.925124
26609,supercritical,20.0,4.254561,2.257672,0.956974
26610,supercritical,20.0,4.489795,2.308584,0.990282
26611,supercritical,20.0,4.738035,2.366805,1.023531


## 🤖 Model Training

## 📈 Analysis